<a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>


###### Ch. 11 동적 계획법(Dynamic Programming)

**동적 계획법**<sub>dynamic programming</sub>이란?<br>

복잡한 문제를 재귀를 통해 간단한 하위 문제로 분류하여 단순화하여 해결하는 방법.

어떤 문제가 **최적 부분 구조**<sub>optimal substructure</sub>와 **중복되는 부분 문제**<sub>overlapping subproblem</sub>를 가지고 있다면, 동적 계획법으로 해결 할 수 있다. 

* 최적 부분 구조 : 답을 구하기 위해서 했던 계산을 반복해야 하는 문제의 구조

* 동적 계획법을 사용하려면, 먼저 최적 부분 구조가 있는지 확인해야 한다. 동적 계획법은 부분 문제를 풀고 결과를 저장한 후, 다음 부분 문제(중복되는 부분 문제)를 푸는 과정에서 저장된 결과를 사용한다.  

# 메모제이션

**메모이제이션**<sub>memoization</sub> 은 프로그램이 동일한 계산을 반복할 때, 이전에 계산한 값을 메모리에 저장하여 동일한 계산의 반복 수행을 제거하여 프로그램의 실행 속도를 빠르게 하는 기법이다. (동적 계획법의 핵심)

## 피보나치 수열 

메몽이제이션 사용 여부에 따른 속도 차이를 보기 위해 함수를 세가지 버전으로 작성해보자. 

In [1]:
from functools import wraps
import time


def benchmark(method):
    @wraps(method)
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        # print("%r: %2.2f ms" % (method.__name__, (te - ts) * 1000))
        # print(f"{method.__name__}: {((te-ts)*1000):.2f} ms")
        print("{0}: {1:0.2f} ms".format(method.__name__, ((te-ts)*1000)))
        return result

    return timed

In [2]:
from functools import wraps

def memo(func):
    cache = {}
    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

def fib(n):
    if n < 2:
        return 1
    
    else:
        return fib(n-1) + fib(n-2)
    

@memo
def fib2(n):
    if n < 2:
        return 1
    
    else:
        return fib2(n-1) + fib2(n-2)
    
    
def fib3(m, n):
    if m[n] == 0:
        m[n] = fib3(m, n-1) + fib3(m, n-2)
    return m[n]

In [3]:
@benchmark
def test_fib(n):
    print(fib(n))

@benchmark
def test_fib2(n):
    print(fib2(n))
    
@benchmark
def test_fib3(n):
    m = [0] * (n + 1)
    m[0], m[1] = 1, 1
    print(fib3(m, n))


In [4]:
n = 35
test_fib(n)
test_fib2(n)
test_fib3(n)


14930352
test_fib: 2032.85 ms
14930352
test_fib2: 0.00 ms
14930352
test_fib3: 0.00 ms


메모이제이션을 사용하면 다음 그림과 같이 반복을 줄일수 있다. 
<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/742bb2c9-f23e-4352-bd2b-cfb0f8c67e67.png' width='400' />

**Note_**데커레이터 함수에 @wraps를 사용하는 경우와 사용하지 않는 경우 결과 차이는 없지만, 데커레이터를 사용하는 함수의 `__name__`과 `__doc__` 값은 차이가 있다. 파이썬 코드에서 데커레이터를 사용한다면, 디버깅을 위해서 functools.wraps 모듈을 사용한다. 다음 코드로 functools.wraps 사용법을 이해해 보자. 스택 오버 플로에 있는 코드를 참조했다. 

In [5]:
# https://stackoverflow.com/questions/308999/what-does-functools-wraps-do
from functools import wraps


def logged(func):
    def with_logging(*args, **kwargs):
        """with_logging() 함수"""
        print(func.__name__ + " 호출")
        return func(*args, **kwargs)
    return with_logging


@logged
def f(x):
    """첫 번째, 데커레이터 사용 """
    return x + x * x


def f2(x):
    """두 번째, 데커레이터 사용 X """
    return x + x * x


def logged2(func):
    @wraps(func)
    def with_logging(*args, **kwargs):
        print(func.__name__ + " 호출")
        return func(*args, **kwargs)
    return with_logging


@logged2
def f3(x):
    """세 번째, wraps와 데커레이터 사용 """
    return x + x * x


if __name__ == "__main__":
    print("결과: {0}".format(f(5)))
    print("__name__: {0}".format(f.__name__))
    print("__doc__: {0}".format(f.__doc__))
    print("-----------------------------")
    f2 = logged(f2)
    print("결과: {0}".format(f2(5)))
    print("__name__: {0}".format(f2.__name__))
    print("__doc__: {0}".format(f2.__doc__))
    print("-----------------------------")
    print("결과: {0}".format(f3(5)))
    print("__name__: {0}".format(f3.__name__))
    print("__doc__: {0}".format(f3.__doc__))

f 호출
결과: 30
__name__: with_logging
__doc__: with_logging() 함수
-----------------------------
f2 호출
결과: 30
__name__: with_logging
__doc__: with_logging() 함수
-----------------------------
f3 호출
결과: 30
__name__: f3
__doc__: 세 번째, wraps와 데커레이터 사용 


# 연습문제

## 최장 증가 부분열

메모제이션의 또 다른 재미있는 예제는 주어진 리스트에서 **최장 증가 부분열**<sub>longest increasing subsequence</sub> 을 찾는 문제이다. 

예를 들어 리스트 `[ 94, 8, 78, 22, 38, 79, 93, 8, 84, 39 ]` 가 있다면,

[ 94, **8**, 78, **22**, **38**, **79**, **93**, 8, 84, 39 ]<br>
[ 94, **8**, 78, **22**, **38**, **79**, 93, 8, **84**, 39 ]
<br>

두가지 경우가 있다. 

In [6]:
from functools import wraps
import time


def benchmark(method):
    @wraps(method)
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        # print("%r: %2.2f ms" % (method.__name__, (te - ts) * 1000))
        # print(f"{method.__name__}: {((te-ts)*1000):.2f} ms")
        print("{0}: {1} ms".format(method.__name__, ((te-ts)*1000)))
        return result

    return timed

In [7]:
from bisect import bisect
from itertools import combinations
from functools import wraps

### 단순한 방법

In [8]:
def naive_logest_inc_subseq(seq):
    for length in range(len(seq), 0, -1):
        for sub in combinations(seq, length):
            if list(sub) == sorted(sub):
                return len(sub)

### 동적 계획법            

In [9]:
def dp_longest_inc_subseq(seq):
    L = [1] * len(seq)
    res = []
    for cur, val in enumerate(seq):
        for pre in range(cur):
            if seq[pre] <= val:
                L[cur] = max(L[cur], 1 + L[pre])
                
    return max(L)

def memo(func):
    cache = {}
    
    @wraps(func)
    def wrap(*args):
        if args not in cache:
            cache[args] = func(*args)
        return cache[args]
    return wrap

### 메모제이션

In [10]:
def memoized_longest_inc_subseq(seq):
    @memo
    def L(cur):
        res = 1
        for pre in range(cur):
            if seq[pre] <= seq[cur]:
                res = max(res, 1 + L(pre))
        return res
    return max(L(i) for i in range(len(seq)))

### 이진 검색

In [11]:
def longest_inc_bisec(seq):
    end = []
    for val in seq:
        idx = bisect(end, val)
        if idx == len(end):
            end.append(val)
        else:
            end[idx] = val
#         print(end)
    return len(end)

In [12]:
@benchmark
def test_naive_logest_inc_subseq():
    print(naive_logest_inc_subseq(s1))
    
@benchmark
def test_dp_longest_inc_subseq():
    print(dp_longest_inc_subseq(s1))

@benchmark
def test_memoized_longest_inc_subseq():
    print(memoized_longest_inc_subseq(s1))
    
@benchmark
def test_longest_inc_bisec():
    print(longest_inc_bisec(s1))

In [13]:
s1 = [ 94, 8, 78, 22, 38, 79, 93, 8, 84, 39 ]
test_naive_logest_inc_subseq()
test_dp_longest_inc_subseq()
test_memoized_longest_inc_subseq()
test_longest_inc_bisec()

5
test_naive_logest_inc_subseq: 0.0 ms
5
test_dp_longest_inc_subseq: 0.0 ms
5
test_memoized_longest_inc_subseq: 0.0 ms
5
test_longest_inc_bisec: 0.0 ms


2.1.2 의 동적 계획법 및 2.1.3 메모제이션의 경우 실행 과정을 그림으로 나타내면 아래와 같다. 

<img src='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/d4954708fd8f4731a091c75bea2b2ea7/7492122d-cc21-4293-8cde-2d48f636c1f0.png' width='600' />

동정 계획법과 메모이제이션의 차이를 요약하자면, 도표를 그리는 방식(일반적인 동적 계획법의 접근 방식)은 상향식<sub>bottom-up</sub> 이고, 메모이제이션(일반적인 재귀 함수의 호출 방식)은 하향식<sub>top-down</sub>이라는 점이다. 

https://stackoverflow.com/questions/6184869/what-is-the-difference-between-memoization-and-dynamic-programming

https://programming.guide/dynamic-programming-vs-memoization-vs-tabulation.html

**Reference**

* <a href='https://github.com/SeWonKwon' ><div> <img src ='https://slid-capture.s3.ap-northeast-2.amazonaws.com/public/image_upload/6556674324ed41a289a354258718280d/964e5a8b-75ad-41fc-ae75-0ca66d06fbc7.png' align='left' /> </div></a>

<br>

* [파이썬 자료구조와 알고리즘, 미아 스타인](https://github.com/AstinCHOI/Python-and-Algorithms-and-Data-Structures)